In [1]:
!pip install -q datasets
!pip install -q transformers==4.30
!pip install -q peft
!pip install -q trl
!pip install -q accelerate -U
!pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from datasets import load_dataset
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig

import torch

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset",  split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
ds = dataset.train_test_split(test_size=0.3)
ds

DatasetDict({
    train: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 18810
    })
    test: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 8062
    })
})

In [51]:
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/llm/fine-tuned-fb-opt-350m",
    dataloader_drop_last=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    logging_steps=5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    fp16=False,
    weight_decay=0.05,
    ddp_find_unused_parameters=False,
    run_name="fb-opt-125",
)

In [52]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoModelForCausalLM
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="fp4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [53]:
from torch import nn

for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float16)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float16)
model.lm_head = CastOutputToFloat(model.lm_head)

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [55]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
model.to(device)

trainable params: 1,572,864 || all params: 332,769,280 || trainable%: 0.472659014678278


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_fea

In [56]:
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

def formatting_func(example):
    text = f"### Question: {example['instruction']}\n  ### Category: {example['category']}\n ### Answer: {example['response']}"
    return text

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds['train'],
    formatting_func=formatting_func,
    seq_length=1024
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds['test'],
    formatting_func=formatting_func,
    seq_length=1024
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=1024,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    packing=True,
)

print("Training...")
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Training...


Epoch,Training Loss,Validation Loss
0,2.303500,2.074219
1,2.014500,1.772461
2,1.826200,1.620117
3,1.696500,1.517578
4,1.623400,1.451172
5,1.612100,1.428711
6,1.550600,1.363281
7,1.486900,1.311523


Epoch,Training Loss,Validation Loss
0,2.303500,2.074219
1,2.014500,1.772461
2,1.826200,1.620117
3,1.696500,1.517578
4,1.623400,1.451172
5,1.612100,1.428711
6,1.550600,1.363281
7,1.486900,1.311523
8,1.494100,1.295898
9,1.454100,1.274414


TrainOutput(global_step=2400, training_loss=1.7723710123697918, metrics={'train_runtime': 11529.7261, 'train_samples_per_second': 16.314, 'train_steps_per_second': 1.36, 'total_flos': 5.37693045915648e+16, 'train_loss': 1.7723710123697918, 'epoch': 9.15})

In [59]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", quantization_config=nf4_config)

# Load the LoRA adaptors
from peft import PeftModel

# Load the Lora model
model = PeftModel.from_pretrained(model, "/content/gdrive/MyDrive/llm/fine-tuned-fb-opt-350m/checkpoint-2400/")
model.eval()
model = model.merge_and_unload()

# Save for future use
model.save_pretrained("/content/gdrive/MyDrive/llm/fine-tuned-fb-opt-350/merged-epoch_5")
# model = model.

`low_cpu_mem_usage` was None, now set to True since model is quantized.
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [70]:
inputs = tokenizer("I want help to cancel my purcahse order 1234", return_tensors="pt").to(device)
generation_output = model.generate(**inputs,
                                    return_dict_in_generate=True,
                                    output_scores=True,
                                    max_length=100,
                                    num_beams=3,
                                    do_sample=True,
                                    repetition_penalty=1.5)

In [71]:
generation_output
print( tokenizer.decode(generation_output['sequences'][0]) )

</s>I want help to cancel my purcahse order 1234 days ago
  ### Category: ORDER
 ### Answer: I'm sorry to hear that you're interested in canceling your order. To assist you with this, could you please provide me with the details of your order? This will help me identify any specific issues or concerns you may have, and allow me to guide you through the process step-by-step. Your satisfaction is our top priority, and we appreciate your patience


In [79]:
inputs = tokenizer("I am waiting for refund of order 1234.", return_tensors="pt").to(device)
generation_output = model.generate(**inputs,
                                    return_dict_in_generate=True,
                                    output_scores=True,
                                    max_length=120,
                                    num_beams=3,
                                    do_sample=True,
                                    repetition_penalty=1.5)

In [80]:
print( tokenizer.decode(generation_output['sequences'][0]) )

</s>I am waiting for refund of order 1234. I'm sorry, but I have to be aware that your order has been approved by the company, and it can't be helped. Please let me know if you have any other questions or concerns. I'm here to assist you every step of the way!</s>


In [83]:
inputs = tokenizer("I have to see the bloody status of the refund, how to do it?", return_tensors="pt").to(device)
generation_output = model.generate(**inputs,
                                    return_dict_in_generate=True,
                                    output_scores=True,
                                    max_length=150,
                                    num_beams=3,
                                    do_sample=True,
                                    repetition_penalty=1.5)
print( tokenizer.decode(generation_output['sequences'][0]) )

</s>I have to see the bloody status of the refund, how to do it?

1. Click on the "Watch Now" button on your account or login page.
2. Click on the "Buy Now" option to get a premium price for your purchase.
3. Click on the "Buy Now" option to get a premium price for your purchase.
4. Click on the "Buy Now" button to get a premium price for your purchase.
5. Click on the "Buy Now" button to get a premium price for your purchase.

If you encounter any difficulties or have any questions, please don't hesitate to reach out. I'm here to assist you every step of the way!</s>


In [ ]:
inputs = tokenizer("I have to see the bloody status of the refund, how to do it?", return_tensors="pt").to(device)
generation_output = model.generate(**inputs,
                                    return_dict_in_generate=True,
                                    output_scores=True,
                                    max_length=150,
                                    num_beams=3,
                                    do_sample=True,
                                    repetition_penalty=1.5)
print( tokenizer.decode(generation_output['sequences'][0]) )